## Class Imbalancing Implement on MNIST dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from imblearn.over_sampling import SMOTE
import numpy as np

### Define the CNN Model

In [2]:
# Define the CNN model for CIFAR-10
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        # Calculate the correct size of the fully connected layer input
        self.fc1 = nn.Linear(64 * 14 * 14, 128)  # Adjusted dimension based on CIFAR-10 input size and CNN structure
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


### Load CIFAR-10 Dataset and Create Dataloaders

In [3]:
# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


### Define Training and Testing Functions

In [4]:
# Train the CNN model on original dataset
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

def test_model(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    inputs, logits = [], []  # Correct initialization
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            # Store input vectors and logits
            inputs.append(data)
            logits.append(output)

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')
    return inputs, logits, accuracy


### Train and Test the Model on the Original Dataset

In [5]:
# Instantiate and train the model
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, criterion, optimizer)
inputs_original, logits_original, original_accuracy = test_model(model, test_loader)


Epoch 1/10, Loss: 0.9575943946838379
Epoch 2/10, Loss: 0.8130965232849121
Epoch 3/10, Loss: 0.567131519317627
Epoch 4/10, Loss: 0.5547571182250977
Epoch 5/10, Loss: 0.31847524642944336
Epoch 6/10, Loss: 0.0833733081817627
Epoch 7/10, Loss: 0.08366585522890091
Epoch 8/10, Loss: 0.09616219997406006
Epoch 9/10, Loss: 0.26953256130218506
Epoch 10/10, Loss: 0.09767892211675644
Test set: Average loss: 2.0636, Accuracy: 0.6755


### Create Imbalanced Dataset

In [6]:
# Function to create imbalanced dataset
def get_imbalanced_data(dataset, num_sample_per_class, shuffle=False, random_seed=0):
    length = dataset.__len__()
    num_sample_per_class = list(num_sample_per_class)
    selected_list = []
    indices = list(range(0, length))

    for i in range(0, length):
        index = indices[i]
        _, label = dataset.__getitem__(index)
        if num_sample_per_class[label] > 0:
            selected_list.append(index)
            num_sample_per_class[label] -= 1

    return selected_list

# Create imbalanced dataset
num_samples_per_class = [5000, 3000, 2000, 1000, 500, 300, 200, 100, 50, 20]
imbalanced_indices = get_imbalanced_data(train_dataset, num_samples_per_class)
imbalanced_dataset = Subset(train_dataset, imbalanced_indices)
imbalanced_loader = DataLoader(imbalanced_dataset, batch_size=64, shuffle=True)


### Apply SMOTE to Balance the Dataset

In [7]:
# Extract data and labels for SMOTE
data = []
labels = []
for d, l in imbalanced_loader:
    data.append(d)
    labels.append(l)
data = torch.cat(data).numpy().reshape(-1, 32*32*3)
labels = torch.cat(labels).numpy()

# Apply SMOTE
smote = SMOTE()
data_resampled, labels_resampled = smote.fit_resample(data, labels)
data_resampled = torch.tensor(data_resampled).reshape(-1, 3, 32, 32).float()
labels_resampled = torch.tensor(labels_resampled)

# Create DataLoader for balanced dataset
balanced_dataset = torch.utils.data.TensorDataset(data_resampled, labels_resampled)
balanced_loader = DataLoader(balanced_dataset, batch_size=64, shuffle=True)


### Train and Test the Model on the Balanced Dataset

In [8]:
# Train the CNN model on balanced dataset
model_balanced = CNN()
optimizer_balanced = optim.Adam(model_balanced.parameters(), lr=0.001)
train_model(model_balanced, balanced_loader, criterion, optimizer_balanced)
inputs_balanced, logits_balanced, balanced_accuracy = test_model(model_balanced, test_loader)

print(f'Original dataset accuracy: {original_accuracy*100}')
print(f'Balanced dataset accuracy after SMOTE: {balanced_accuracy*100}')


Epoch 1/10, Loss: 0.43751591444015503
Epoch 2/10, Loss: 0.011142255738377571
Epoch 3/10, Loss: 0.011839285492897034
Epoch 4/10, Loss: 0.05675537884235382
Epoch 5/10, Loss: 0.000913694326300174
Epoch 6/10, Loss: 0.0014481820398941636
Epoch 7/10, Loss: 7.333533721975982e-05
Epoch 8/10, Loss: 0.05012133717536926
Epoch 9/10, Loss: 0.013388552702963352
Epoch 10/10, Loss: 0.00012767294538207352
Test set: Average loss: 10.3185, Accuracy: 0.3856
Original dataset accuracy: 67.55
Balanced dataset accuracy after SMOTE: 38.56


### Print Input Vectors and Logits

In [9]:
# Print input vectors and logits
print("\nInput Vectors and Logits for Original Dataset:")
for i in range(len(inputs_original)):
    print(f"Batch {i+1}:")
    print("Inputs:", inputs_original[i])
    print("Logits:", logits_original[i])

print("\nInput Vectors and Logits for Balanced Dataset:")
for i in range(len(inputs_balanced)):
    print(f"Batch {i+1}:")
    print("Inputs:", inputs_balanced[i])
    print("Logits:", logits_balanced[i])



Input Vectors and Logits for Original Dataset:
Batch 1:
Inputs: tensor([[[[ 0.2392,  0.2471,  0.2941,  ...,  0.0745, -0.0118, -0.0902],
          [ 0.1922,  0.1843,  0.2471,  ...,  0.0667, -0.0196, -0.0667],
          [ 0.1843,  0.1843,  0.2392,  ...,  0.0902,  0.0196, -0.0588],
          ...,
          [-0.4667, -0.6706, -0.7569,  ..., -0.7020, -0.8980, -0.6863],
          [-0.5216, -0.6157, -0.7255,  ..., -0.7961, -0.7725, -0.8431],
          [-0.5765, -0.5608, -0.6471,  ..., -0.8118, -0.7333, -0.8353]],

         [[-0.1216, -0.1294, -0.0902,  ..., -0.2549, -0.2863, -0.3333],
          [-0.1216, -0.1373, -0.1059,  ..., -0.2549, -0.2863, -0.3098],
          [-0.1373, -0.1451, -0.1294,  ..., -0.2314, -0.2549, -0.3020],
          ...,
          [-0.0275, -0.2157, -0.3098,  ..., -0.2392, -0.4980, -0.3333],
          [-0.0902, -0.2000, -0.3333,  ..., -0.3569, -0.3569, -0.4980],
          [-0.1608, -0.1765, -0.3020,  ..., -0.3961, -0.3412, -0.4745]],

         [[-0.6157, -0.6314, -0.6000,